##### DNN  CNN
    - 독립적인 정보
    - 입력(x) 간의 순서나 연관성을 고려하지 않는다
##### 시계열 : 시간의 연속적인 흐름
    - 시계열 데이터 : 날씨, 주식, 문장 --> 순서가 중요한 데이터    
##### RNN
    - 순환하는 구조
        - 시점1 (월요일):맑음(x1) 정보가 RNN에 들어온다 -> RNN 날씨가 맑았음(h1) 이라는 요약본을생성
        - 시점2 (화요일):흐림(x2) ->RNN 새로운정보(흐림,x2) + 어제의기억(맑았음, h1) 함께 고려 해서
            어제 맑았는데 오늘 흐림 h2 이라는 새로운 요약본을 생성
        - 시점3 (수요일):비(x3)->RNN 새로운정보(비,x3) 정보와  + 어제의기억(어제 맑았는데 오늘 흐림 h2 )
            새로운 상태 h3
        - 반복
    - 알고리즘
        - 각 시점(time step)에서  1.현재의 입력 과 2 과거의기억(hidden state ht-1) 받아서 
        3. 현재의 결과물과  4 다음시점으로 넘겨줄 최신기억 ht 을 생성
        - ht 기억이 시계열 데이터의 맥락(Context) 저장하는 역활
    - 장점
        - 순서가 있는 데이터의 맥락을 학습
    - 한계
        - 기억력이 생각보다 짧다
        - 시계열 데이터가 길어지면(예 100단계전의 정보)
        - 이전정보가 소실되거나 반대로 너무 강해져서 폭주가 되서 제대로 학습이 안된다. 
        - 장기 기억 의존성 문제( Long-Term Dependency Problem)
##### LSTM & GRU
    - LSTM(Long Short-Term Memory) : RNN 내부에 게이트(Gate) 복잡한 장치 -> 잊고, 기억할 정보를 관리
    - GRU(Gated Recurrent Unit) : LSTM 구조를 좀더  단순화시킨 모델, LSTM성능은 비슷, 속도는 빠르다
##### RNN 핵심수식
    - 은닉상태 계산
        - h1 = tanh(wxht-1 + wxt - bh)
    - 출력계산

In [ ]:
import pandas as pd
url = 'https://raw.githubusercontent.com/pia222sk20/python/refs/heads/main/data/time_data_train.csv'

df = pd.read_csv(url)
df.head()

,Date,Open,High,Low,Volume,Close
0,2015-12-16,120,123,118,13181000,123
1,2015-12-17,124,126,122,17284900,123
2,2015-12-18,121,122,118,17948100,118
3,2015-12-21,120,120,116,11670000,117
4,2015-12-22,117,117,115,9689000,116


In [4]:
# 전체 데이터셋 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 967 entries, 0 to 966
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    967 non-null    object
 1   Open    967 non-null    int64 
 2   High    967 non-null    int64 
 3   Low     967 non-null    int64 
 4   Volume  967 non-null    int64 
 5   Close   967 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 45.5+ KB


In [ ]:
# EDA - 탐색적 데이터 분석

In [25]:
# 데이터셋 정의
import numpy as np
from torch.utils.data.dataset import Dataset
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
class StockDataSet(Dataset):
    def __init__(self):
        url = 'https://raw.githubusercontent.com/pia222sk20/python/refs/heads/main/data/time_data_train.csv'
        self.csv = pd.read_csv(url)
        data = self.csv.iloc[: , 1:-1].values
        label = self.csv.iloc[:,-1].values.reshape(-1,1)
        self.data = StandardScaler().fit_transform( data)
        # 정답이 숫자 크다면 정규화가 학습에 도움이된다.
        self.label = StandardScaler().fit_transform(label)
        self.data = torch.Tensor(self.data)
        self.label = torch.Tensor(self.label)
    def __len__(self):
        return len(self.data) - 30  # 사용가능한 배치 개수
    def __getitem__(self, index):
        data = self.data[index: index + 30]
        label = self.label[index + 30]
        return data, label


In [ ]:
import torch.nn as nn
class StockRNN(nn.Module):
    def __init__(self):
        super(StockRNN,self).__init__()
        # (30일, 배치16개 , 각 입력의 특성 4개)  batch_first = False
        # (배치16개, 30일 각 입력의 특성 4) batch_first = True
        self.rnn = nn.RNN(input_size=+4, hidden_size=8,num_layers=5,batch_first=True)
        # 출력 (batch, 30 , 8)
        self.fc1 = nn.Linear(30*8,64)
        self.fc2 = nn.Linear(64,1)
        self.relu = nn.ReLU()
    def forward(self,x,ho):
        x,hn =  self.rnn(x,ho)


(tensor([[-0.9954, -0.9828, -0.9983,  0.5097],
         [-0.9571, -0.9545, -0.9593,  1.1445],
         [-0.9858, -0.9923, -0.9983,  1.2471],
         [-0.9954, -1.0111, -1.0178,  0.2759],
         [-1.0242, -1.0394, -1.0276, -0.0305],
         [-1.0242, -1.0206, -1.0178, -0.0869],
         [-1.0146, -1.0206, -1.0081, -0.9831],
         [-1.0242, -1.0394, -1.0373, -0.2262],
         [-1.0146, -1.0111, -1.0081, -0.2672],
         [-1.0050, -1.0206, -1.0178, -0.2686],
         [-1.0337, -1.0394, -1.0373, -0.0949],
         [-1.1008, -1.1055, -1.1251,  1.6875],
         [-1.0912, -1.0961, -1.1154,  1.2033],
         [-1.1391, -1.0300, -1.1251,  3.5826],
         [-1.0337, -0.9923, -1.0569,  3.6741],
         [-1.0337, -1.0300, -1.0666,  1.2655],
         [-1.0721, -1.0394, -1.0666,  1.8616],
         [-1.0337, -1.0300, -1.0276,  0.8117],
         [-1.0529, -1.0678, -1.1251,  2.3259],
         [-1.1295, -1.1149, -1.1642,  2.1315],
         [-1.1678, -1.1432, -1.1544,  1.5298],
         [-1.